In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

As a company operating in the online fashion sector, DiCo needs to evaluate sales performance and understand the most frequently purchased fashion items. Apart from that, he also needs to understand his customers better so he could create a more efficient campaign strategy.

# Several Business Questions

1. How has the company's sales and revenue performance been in the last few months?
2. What's products sell the most and least?
3. When was the customer's last transaction made?
4. How frequently has a customer made a purchase in the last few months?
5. How much money did the custmoers spend in the last few months?

# Getting the Data

In [2]:
customers_df = pd.read_csv("../1-data_wrangling/clean_customers_df.csv")
orders_df = pd.read_csv("../1-data_wrangling/clean_orders_df.csv")
products_df = pd.read_csv("../1-data_wrangling/clean_products_df.csv")
sales_df = pd.read_csv("../1-data_wrangling/clean_sales_df.csv")

# Start the EDA

## customer_df

### Statistical parameters of customer_df 

In [3]:
customers_df.describe(include="all")

,customer_id,customer_name,gender,age,home_address,zip_code,city,state,country
count,1000.000000,1000,1000,1000.000000,1000,1000.000000,1000,1000,1000
unique,NaN,1000,3,NaN,1000,NaN,961,8,1
top,NaN,fulan 1,Prefer not to say,NaN,8606 Victoria TerraceSuite 560,NaN,East Sophia,South Australia,Australia
freq,NaN,1,742,NaN,1,NaN,3,139,1000
mean,499.420000,NaN,NaN,49.860000,NaN,5004.872000,NaN,NaN,NaN
std,289.420676,NaN,NaN,17.647828,NaN,2884.497332,NaN,NaN,NaN
min,1.000000,NaN,NaN,20.000000,NaN,2.000000,NaN,NaN,NaN
25%,248.750000,NaN,NaN,34.000000,NaN,2401.750000,NaN,NaN,NaN
50%,498.500000,NaN,NaN,50.000000,NaN,5083.000000,NaN,NaN,NaN
75%,750.250000,NaN,NaN,65.000000,NaN,7460.250000,NaN,NaN,NaN


We can see that the distribution of our customers' ages as follows:
1. It ranges from 20 to 80 years old.
2. The average age of our customer is 49.86 years old
3. The Standard Deviation is 17.64

These informations give an idea that our customer segmentation is quite broad, from teenager to the eldery

### Customer demographic based on gender

In [4]:
customers_df.groupby(by="gender").agg({
    "customer_id": "nunique",
    "age": ["max","min","mean","std"]
})

customer_id age                          
                      nunique max min       mean        std
gender                                                     
Female                    115  79  20  49.147826  16.646607
Male                      143  80  20  51.230769  18.462635
Prefer not to say         741  80  20  49.706199  17.648386

The customer we have is dominated by "prefer not to say". Howeever, the age distributions of each gender are quite similiar, ranging between 20 to 80

### Distribution of customers by city and state

#### By City

In [5]:
customers_df.groupby(by="city")["customer_id"].nunique().sort_values(ascending=False)

city
East Aidan       3
East Sophia      3
New Ava          3
West Jackfort    2
Corkeryshire     2
                ..
Jordanside       1
Josephland       1
Josephmouth      1
Justinport       1
Zacville         1
Name: customer_id, Length: 961, dtype: int64

#### By State

In [6]:
customers_df.groupby(by="state")["customer_id"].nunique().sort_values(ascending=False)

state
South Australia                 139
Queensland                      134
New South Wales                 132
Northern Territory              125
Western Australia               123
Australian Capital Territory    121
Victoria                        121
Tasmania                        104
Name: customer_id, dtype: int64

By examining at these values, we can determine that the majority of our customers are located in the cities of East Aidan, Easy Sophia, and New Ava, with three customers in each city

## order_df

In [7]:
orders_df.tail()

,order_id,customer_id,payment,order_date,delivery_date
995,996,345,37843,2021-01-13,2021-02-02
996,997,346,53831,2021-01-18,2021-01-31
997,998,407,53308,2021-05-05,2021-05-21
998,999,428,31643,2021-06-15,2021-07-12
999,1000,896,27836,2021-04-07,2021-04-24


In [8]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   order_id       1000 non-null   int64 
 1   customer_id    1000 non-null   int64 
 2   payment        1000 non-null   int64 
 3   order_date     1000 non-null   object
 4   delivery_date  1000 non-null   object
dtypes: int64(3), object(2)
memory usage: 39.2+ KB


This table contains various information related to an order consisting of order_id, customer_id, order_date, delivery_date. based on this information, we could create a new column that contains the delivery time information. We will use the `total_seconds()` method and then divided by `86400` (seconds in a day). We need to convert the order_date and delivery_date into datetime datatype

### Convert the order_date and delivery_date datatypes to datetime

In [9]:
for col in orders_df.columns[-2:]:
    orders_df[col] = pd.to_datetime(orders_df[col])

orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       1000 non-null   int64         
 1   customer_id    1000 non-null   int64         
 2   payment        1000 non-null   int64         
 3   order_date     1000 non-null   datetime64[ns]
 4   delivery_date  1000 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(3)
memory usage: 39.2 KB


### Create a new column named delivery_time

In [10]:
seconds_a_day = 86400
orders_df["delivery_time"] = round((orders_df["delivery_date"] - orders_df["order_date"]).apply(lambda x: x.total_seconds()) / seconds_a_day)

In [16]:
orders_df[orders_df["order_id"] == 717]

,order_id,customer_id,payment,order_date,delivery_date,delivery_time
716,717,696,52327,2021-03-11,2021-04-05,25.0


In [18]:
orders_df.describe(include="all")

,order_id,customer_id,payment,order_date,delivery_date,delivery_time
count,1000.000000,1000.000000,1000.000000,1000,1000,1000.000000
mean,500.500000,506.640000,33972.936000,2021-05-27 18:38:52.800000,2021-06-10 20:31:12,14.078000
min,1.000000,1.000000,10043.000000,2021-01-01 00:00:00,2021-01-03 00:00:00,1.000000
25%,250.750000,275.250000,21329.250000,2021-03-13 18:00:00,2021-03-28 00:00:00,8.000000
50%,500.500000,515.000000,33697.500000,2021-05-27 12:00:00,2021-06-11 12:00:00,14.000000
75%,750.250000,737.250000,46249.000000,2021-08-12 06:00:00,2021-08-24 06:00:00,21.000000
max,1000.000000,1000.000000,59910.000000,2021-10-24 00:00:00,2021-11-20 00:00:00,27.000000
std,288.819436,277.115502,14451.609047,NaN,NaN,7.707225


Based on these results. It seems that the average of delivery time is 14 days with maximum 27 days and minimum 1 day

## Exploration between customers_df and orders_df

There is a column that contains informations about customer_id who have placed orders. We can use this information to identify customers who have never placed an order. We can create a new column called status that consisting of two values: 
1. "Active":  Customer that has placed an order at least once
2. "Inactive" : Customer that hasn't placed an order at all

### Create a new column on customer_id named status

In [21]:
customer_id_in_orders_df = orders_df["customer_id"].tolist() # convert to a list
customers_df["status"] = customers_df["customer_id"].apply(lambda x: "Active" if x in customer_id_in_orders_df else "Inactive")
customers_df.sample(5)

,customer_id,customer_name,gender,age,home_address,zip_code,city,state,country,status
66,67,fulan 67,Male,59,928 Johnson View RdApt. 583,5630,Lucamouth,South Australia,Australia,Inactive
25,26,fulan 26,Prefer not to say,50,499 Darcy CrestApt. 192,8816,New Oliverside,New South Wales,Australia,Active
896,897,fulan 897,Male,76,8325 Molly ParkwaySuite 440,1540,Dickinsonside,Queensland,Australia,Inactive
256,257,fulan 257,Prefer not to say,61,9347 Morrison RoadSuite 865,1166,Johnstonshire,Australian Capital Territory,Australia,Active
660,661,fulan 661,Male,45,4317 Holly CourtSuite 716,1431,North Beau,Western Australia,Australia,Active


### Look at the customer demographic based on status

In [23]:
customers_df.groupby("status")["customer_id"].count()

status
Active      618
Inactive    382
Name: customer_id, dtype: int64

This is certainly bad news because almost 30% of our customers have never placed an order before

To obtain more information regarding these two data, we need to combine them through a join or  merge process 

### Merge orders_df and customers_df

In [25]:
orders_customers_df = pd.merge (
    left = orders_df,
    right = customers_df,
    how = "left",
    left_on = "customer_id",
    right_on = "customer_id",
)
orders_customers_df.tail()

,order_id,customer_id,payment,order_date,delivery_date,delivery_time,customer_name,gender,age,home_address,zip_code,city,state,country,status
998,996,345,37843,2021-01-13,2021-02-02,20.0,fulan 345,Prefer not to say,62.0,9182 Hermann MallApt. 890,799.0,Audreyfurt,Victoria,Australia,Active
999,997,346,53831,2021-01-18,2021-01-31,13.0,fulan 346,Prefer not to say,40.0,59 Bergstrom GroveSuite 260,6060.0,Olsonbury,Northern Territory,Australia,Active
1000,998,407,53308,2021-05-05,2021-05-21,16.0,fulan 407,Male,41.0,5233 Lara AvenueSuite 291,8697.0,Hartmannshire,Queensland,Australia,Active
1001,999,428,31643,2021-06-15,2021-07-12,27.0,fulan 428,Prefer not to say,66.0,80 Hermiston WaySuite 403,1112.0,North Liamfort,Australian Capital Territory,Australia,Active
1002,1000,896,27836,2021-04-07,2021-04-24,17.0,fulan 896,Prefer not to say,67.0,985 Nguyen TerraceApt. 362,7737.0,Ryanberg,South Australia,Australia,Active


#### Exploring the number of orders by city

In [32]:
orders_customers_df.groupby(by="city")["order_id"].nunique().sort_values(ascending=False).reset_index().head(10)

,city,order_id
0,Jordanside,6
1,New Ava,6
2,West Kai,5
3,O'keefeton,5
4,East Max,5
5,Lake Rose,5
6,Rubyfort,5
7,Port Hannahburgh,5
8,Kiehnfurt,4
9,Lake Jesse,4


based on these results, it is known that the cities of Jordanside and New Ava are the two cities that have the largest number of orders.

#### Exploring the number of orders by state

In [30]:
orders_customers_df.groupby(by="state")["order_id"].nunique().sort_values(ascending=False).reset_index()

,state,order_id
0,South Australia,148
1,Queensland,139
2,New South Wales,130
3,Western Australia,129
4,Australian Capital Territory,118
5,Victoria,118
6,Tasmania,112
7,Northern Territory,106


#### Exploring the number of orders by gender

In [29]:
orders_customers_df.groupby(by="gender")["order_id"].nunique().sort_values(ascending=False).reset_index()

,gender,order_id
0,Prefer not to say,725
1,Female,139
2,Male,136


#### Exploring the number of orders by age_group

In [33]:
orders_customers_df["age_group"] = orders_customers_df["age"].apply(lambda x: "Youth" if x <= 24 else ("Seniors" if x > 64 else "Adults") )

In [34]:
orders_customers_df.groupby(by="age_group")["order_id"].nunique().sort_values(ascending=False)

age_group
Adults     682
Seniors    227
Youth       93
Name: order_id, dtype: int64

## Exploration of product_df and sales_df

In [35]:
products_df.describe(include="all")

,product_id,product_type,product_name,size,colour,price,quantity,description
count,1260.000000,1260,1260,1260,1260,1260.000000,1260.000000,1260
unique,NaN,3,35,8,7,NaN,NaN,1260
top,NaN,Shirt,Denim,XS,red,NaN,NaN,"A red coloured, XS sized, Oxford Cloth Shirt"
freq,NaN,420,70,252,180,NaN,NaN,1
mean,629.500000,NaN,NaN,NaN,NaN,105.805556,60.150000,NaN
std,363.874979,NaN,NaN,NaN,NaN,9.704423,11.670573,NaN
min,0.000000,NaN,NaN,NaN,NaN,90.000000,40.000000,NaN
25%,314.750000,NaN,NaN,NaN,NaN,95.750000,50.000000,NaN
50%,629.500000,NaN,NaN,NaN,NaN,108.500000,60.000000,NaN
75%,944.250000,NaN,NaN,NaN,NaN,114.000000,70.000000,NaN


In [36]:
sales_df.describe(include="all")

,sales_id,order_id,product_id,price_per_unit,quantity,total_price
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,2499.500000,503.038200,634.053200,103.501600,1.99240,206.360000
std,1443.520003,285.964418,363.255794,9.195004,0.80751,86.357457
min,0.000000,1.000000,1.000000,90.000000,1.00000,90.000000
25%,1249.750000,258.000000,323.000000,95.000000,1.00000,112.000000
50%,2499.500000,504.500000,635.000000,102.000000,2.00000,204.000000
75%,3749.250000,749.000000,951.000000,112.000000,3.00000,285.000000
max,4999.000000,999.000000,1259.000000,119.000000,3.00000,357.000000


Based on these results. The goods sold in range 90-119 dollars. Also we observed the maximum total_prices for a customer's transactions was 357 dollars.

### Find out the most expensive and cheapest product prices

In [47]:
products_df.sort_values(by="price", ascending=False)

,product_id,product_type,product_name,size,colour,price,quantity,description
698,698,Jacket,Parka,L,violet,119,53,"A violet coloured, L sized, Parka Jacket"
665,665,Jacket,Parka,XS,red,119,65,"A red coloured, XS sized, Parka Jacket"
692,692,Jacket,Parka,M,indigo,119,66,"A indigo coloured, M sized, Parka Jacket"
693,693,Jacket,Parka,L,indigo,119,44,"A indigo coloured, L sized, Parka Jacket"
694,694,Jacket,Parka,XL,indigo,119,78,"A indigo coloured, XL sized, Parka Jacket"
...,...,...,...,...,...,...,...,...
1219,1219,Trousers,Pleated,XL,indigo,90,45,"A indigo coloured, XL sized, Pleated Trousers"
1220,1220,Trousers,Pleated,XS,violet,90,58,"A violet coloured, XS sized, Pleated Trousers"
1221,1221,Trousers,Pleated,S,violet,90,50,"A violet coloured, S sized, Pleated Trousers"
1222,1222,Trousers,Pleated,M,violet,90,45,"A violet coloured, M sized, Pleated Trousers"


As we can see if Parka is the most expensive one and Bomber is the cheapest

### Search for product-related information based on product type and name

In [49]:
products_df.groupby(["product_type", "size"]).agg({
    "product_id": "nunique",
    "quantity": "sum",
    "price": ["min", "max"]
})

product_id quantity price     
                                  nunique      sum   min  max
product_type product_name size                               
Jacket       Bomber       L             6      372    90   90
                          M             7      421    90   90
                          S             7      377    90   90
                          XL            7      410    90   90
                          XS            7      439    90   90
...                                   ...      ...   ...  ...
Trousers     Wool         L             7      399   111  111
                          M             7      376   111  111
                          S             7      395   111  111
                          XL            7      366   111  111
                          XS            7      439   111  111

[186 rows x 4 columns]

In [51]:
products_df.groupby("product_name").agg({
    "product_id": "nunique",
    "quantity": "sum",
    "price": ["min", "max"]
})

product_id quantity price     
                     nunique      sum   min  max
product_name                                    
Bomber                    35     2083    90   90
Camp Collared             35     2071   112  112
Cardigan                  35     2032   118  118
Cargo Pants               35     2146   106  106
Casual Slim Fit           35     2086   113  113
Chambray                  35     2020   105  105
Chinos                    35     2101   100  100
Coach                     35     2158   115  115
Cords                     35     2260   113  113
Cropped                   35     2085    99   99
Cuban Collar              35     2017    93   93
Denim                     70     4204    92  115
Drawstring                35     2042   104  104
Dress                     35     2125   119  119
Flannel                   35     2135    96   96
Henley                    35     2051    92   92
High-Waisted              35     2198    98   98
Joggers                   35     2107    94   94
Leather                   35     2276   113  113
Linen                     35     2138   116  116
Mandarin Collar           35     2203   108  108
Oxford Cloth              35     2071   114  114
Parka                     35     2201   119  119
Peacoat                   35     2067   102  102
Pleated                   35     2030    90   90
Polo                      35     2196   117  117
Puffer                    35     1946   110  110
Pullover                  35     2184   114  114
Relaxed Leg               35     2002    95   95
Shearling                 35     2169    95   95
Slim-Fit                  35     2155   119  119
Tracksuit Bottoms         35     2038    91   91
Trench Coat               35     2132   112  112
Windbreaker               35     2085   109  109
Wool                      35     1975   111  111

The pivot table above can give us an idea of ​​the products sold by Dicoding Collection. We need to find out what's the best-selling products. First, we need to merge the product_df and sales_df

In [52]:
sales_products_df = pd.merge(
    left=sales_df,
    right=products_df,
    how="left",
    left_on="product_id",
    right_on="product_id",
)

sales_products_df.head()

,sales_id,order_id,product_id,price_per_unit,quantity_x,total_price,product_type,product_name,size,colour,price,quantity_y,description
0,0,1,218,106,2,212,Shirt,Chambray,L,orange,105,44,"A orange coloured, L sized, Chambray Shirt"
1,1,1,481,118,1,118,Jacket,Puffer,S,indigo,110,62,"A indigo coloured, S sized, Puffer Jacket"
2,2,1,2,96,3,288,Shirt,Oxford Cloth,M,red,114,54,"A red coloured, M sized, Oxford Cloth Shirt"
3,3,1,1002,106,2,212,Trousers,Wool,M,blue,111,52,"A blue coloured, M sized, Wool Trousers"
4,4,1,691,113,3,339,Jacket,Parka,S,indigo,119,53,"A indigo coloured, S sized, Parka Jacket"


#### See the product sales information based on type

In [54]:
sales_products_df.groupby("product_type").agg({
    "sales_id": "nunique",
    "quantity_x": "sum",
    "total_price":"sum"
})

,sales_id,quantity_x,total_price
product_type,,,
Jacket,1676,3343,357026
Shirt,1641,3259,333600
Trousers,1683,3360,341174


Trousers are the best-selling product type. However, if we look at the revenue received, Jackets are the type of product that contributes the most to the company's revenue. 

#### See the product sales information based on name

In [56]:
sales_products_df.groupby("product_name").agg({
    "sales_id": "nunique",
    "quantity_x": "sum",
    "total_price":"sum"
}).sort_values(by="total_price", ascending=False)

,sales_id,quantity_x,total_price
product_name,,,
Denim,273,527,52399
Casual Slim Fit,154,306,36414
Trench Coat,146,299,35581
Shearling,150,302,35334
Puffer,140,298,35164
Flannel,141,281,33158
Cropped,135,284,32660
Pleated,147,308,32340
Joggers,164,334,31062


Denim products are the best-selling products and also contribute the most revenue to the company.  

## Data Exploration all_df

In [58]:
all_df = pd.merge(
    left=sales_products_df,
    right=orders_customers_df,
    how="left",
    left_on="order_id",
    right_on="order_id"
)

all_df.tail()

,sales_id,order_id,product_id,price_per_unit,quantity_x,total_price,product_type,product_name,size,colour,...,customer_name,gender,age,home_address,zip_code,city,state,country,status,age_group
5009,4995,998,321,109,2,218,Shirt,Mandarin Collar,S,orange,...,fulan 407,Male,41.0,5233 Lara AvenueSuite 291,8697.0,Hartmannshire,Queensland,Australia,Active,Adults
5010,4996,998,251,95,3,285,Shirt,Polo,S,orange,...,fulan 407,Male,41.0,5233 Lara AvenueSuite 291,8697.0,Hartmannshire,Queensland,Australia,Active,Adults
5011,4997,999,872,113,1,113,Trousers,Chinos,M,violet,...,fulan 428,Prefer not to say,66.0,80 Hermiston WaySuite 403,1112.0,North Liamfort,Australian Capital Territory,Australia,Active,Seniors
5012,4998,999,998,106,2,212,Trousers,Wool,L,green,...,fulan 428,Prefer not to say,66.0,80 Hermiston WaySuite 403,1112.0,North Liamfort,Australian Capital Territory,Australia,Active,Seniors
5013,4999,999,1105,115,1,115,Trousers,Cropped,XS,blue,...,fulan 428,Prefer not to say,66.0,80 Hermiston WaySuite 403,1112.0,North Liamfort,Australian Capital Territory,Australia,Active,Seniors


### Look at the purchasing preferences based on customer's state and product type

In [59]:
all_df.groupby(["state", "product_type"]).agg({
    "quantity_x": "sum",
    "total_price": "sum"
})

quantity_x  total_price
state                        product_type                         
Australian Capital Territory Jacket               406        43204
                             Shirt                396        40448
                             Trousers             454        46790
New South Wales              Jacket               454        48355
                             Shirt                433        44170
                             Trousers             392        39766
Northern Territory           Jacket               365        38991
                             Shirt                336        33865
                             Trousers             384        38998
Queensland                   Jacket               499        53511
                             Shirt                417        42506
                             Trousers             453        46045
South Australia              Jacket               461        49090
                             Shirt                509        52685
                             Trousers             455        46041
Tasmania                     Jacket               412        44370
                             Shirt                387        39668
                             Trousers             379        38057
Victoria                     Jacket               359        38203
                             Shirt                390        40206
                             Trousers             414        41948
Western Australia            Jacket               396        42380
                             Shirt                397        40639
                             Trousers             433        43936

### Look at the purchasing preferences based on customer's gender and product type

In [65]:
all_df.groupby(["gender", "product_type"]).agg({
    "quantity_x": "sum",
    "total_price": "sum",
})

quantity_x  total_price
gender            product_type                         
Female            Jacket               481        50963
                  Shirt                445        45146
                  Trousers             481        49295
Male              Jacket               480        50978
                  Shirt                410        41939
                  Trousers             453        46071
Prefer not to say Jacket              2391       256163
                  Shirt               2410       247102
                  Trousers            2430       246215

### Look at the purchasing preferences based on customer's age_group and product type

In [66]:
all_df.groupby(["age_group", "product_type"]).agg({
    "quantity_x": "sum",
    "total_price": "sum",
})

quantity_x  total_price
age_group product_type                         
Adults    Jacket              2299       245813
          Shirt               2232       228513
          Trousers            2273       231367
Seniors   Jacket               783        83680
          Shirt                737        75374
          Trousers             770        77878
Youth     Jacket               274        29012
          Shirt                301        30842
          Trousers             322        32432